# Informações preliminares

Este programa utiliza aprendizado por reforço, mais especificamente a metodologia "epsilon-greedy" para simular operações com ações na bolsa de valores. Os dados reais utilizados são baixados via API (link: https://www.alphavantage.co/documentation/) para treinar e testar o agente. Neste caso usa-se as primeiras cotações para treinar o modelo e as demais para testá-lo. Numa verdadeira aplicação prática bastaria apenas mudar o programa para usar treinar o agente com todos os dados e aplicar o modelo a dados futuros, os quais também seriam baixados via API. O tempo seria gerenciado por meio das bibiotecas time e datetime. Por se tratar de um modelo simples com poucos dados de treino (a fim de favorecer uma execução rápida, adequada ao portifólio) o modelo pode falhar e apresentar tanto lucro como prejuízo devido a alta complexidade da bolsa de valores.

# Importação de bibliotecas e módulos

In [59]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from datetime import datetime
import random
import requests
import json
from pprint import pprint

# Fazendo requisição via API para obter arquivo json com as cotações da IBM

In [60]:
from chave import chave_api
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey='+'chave'
r = requests.get(url)
tabela = r.json()

pprint(tabela)

{'Meta Data': {'1. Information': 'Intraday (5min) open, high, low, close '
                                 'prices and volume',
               '2. Symbol': 'IBM',
               '3. Last Refreshed': '2025-04-29 19:55:00',
               '4. Interval': '5min',
               '5. Output Size': 'Compact',
               '6. Time Zone': 'US/Eastern'},
 'Time Series (5min)': {'2025-04-29 11:20:00': {'1. open': '238.1465',
                                                '2. high': '238.6100',
                                                '3. low': '238.1300',
                                                '4. close': '238.5200',
                                                '5. volume': '45972'},
                        '2025-04-29 11:25:00': {'1. open': '238.7200',
                                                '2. high': '239.0150',
                                                '3. low': '238.5100',
                                                '4. close': '238.9400',
          

# Construindo um dataframe a partir do aequivo json baixado via API

In [61]:
df = pd.DataFrame(tabela['Time Series (5min)'])
data = df.T
data = data.reset_index()
data.columns = ['day','open','high','low','close','volume']
data
#data.drop(columns=['timestamp'],axis=1)


,day,open,high,low,close,volume
0,2025-04-29 19:55:00,239.2900,239.2900,239.2900,239.2900,5
1,2025-04-29 19:50:00,238.5700,238.9400,237.2500,237.6700,144
2,2025-04-29 19:45:00,237.8100,237.8100,237.8100,237.8100,30
3,2025-04-29 19:40:00,237.8100,237.8100,237.8100,237.8100,1
4,2025-04-29 19:35:00,238.0000,238.0000,238.0000,238.0000,97
...,...,...,...,...,...,...
95,2025-04-29 11:40:00,238.3700,239.0000,238.3024,238.9200,27946
96,2025-04-29 11:35:00,238.0800,238.4900,237.9401,238.3696,44747
97,2025-04-29 11:30:00,238.9400,238.9700,238.1400,238.2200,26817
98,2025-04-29 11:25:00,238.7200,239.0150,238.5100,238.9400,31416


# Ajustando os formatos do dataframe e reoordenando as linhas em ordem crescente de tempo

In [62]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   day     100 non-null    object
 1   open    100 non-null    object
 2   high    100 non-null    object
 3   low     100 non-null    object
 4   close   100 non-null    object
 5   volume  100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB


In [63]:
data['open'] = data['open'].astype(float)
data['high'] = data['high'].astype(float)
data['low'] = data['low'].astype(float)
data['close'] = data['close'].astype(float)
data['volume'] = data['volume'].astype(float)

In [64]:
data['day'] = pd.to_datetime(data['day'],format="%Y-%m-%d %H:%M:%S")

In [65]:
data = data.sort_values(by='day',ascending=True)
data = data.reset_index(drop=True)
data

,day,open,high,low,close,volume
0,2025-04-29 11:20:00,238.1465,238.610,238.1300,238.5200,45972.0
1,2025-04-29 11:25:00,238.7200,239.015,238.5100,238.9400,31416.0
2,2025-04-29 11:30:00,238.9400,238.970,238.1400,238.2200,26817.0
3,2025-04-29 11:35:00,238.0800,238.490,237.9401,238.3696,44747.0
4,2025-04-29 11:40:00,238.3700,239.000,238.3024,238.9200,27946.0
...,...,...,...,...,...,...
95,2025-04-29 19:35:00,238.0000,238.000,238.0000,238.0000,97.0
96,2025-04-29 19:40:00,237.8100,237.810,237.8100,237.8100,1.0
97,2025-04-29 19:45:00,237.8100,237.810,237.8100,237.8100,30.0
98,2025-04-29 19:50:00,238.5700,238.940,237.2500,237.6700,144.0


# Gráfico de velas

In [66]:
fig =go.Figure(go.Candlestick(x = data['day'],open = data['open'],
               high = data['high'],low = data['low'],
               close = data['close'])).update_layout(template='plotly_dark')
fig.update_layout(title_text='Cotação da IBM em função do tempo',title_font=dict(size=30),title_x=0.5)
fig.update_xaxes(title_text="Tempo",title_font=dict(size=20))
fig.update_yaxes(title_text="Valor da ação",title_font=dict(size=20))
fig.show()

# Separando os preços iniciais para treinar o agente e os seguintes para testar o modelo

In [67]:
df = data['close']
precos_treino = df[0:75]  # dados de treino
precos = df[75:].reset_index(drop=True) # dados de teste

# Definindo os parâmetros iniciais e hiperparâmetros do modelo

In [68]:
# Hiperparâmetros
num_episodios = 1000  #número de episódios para treinar o agente
alpha = 0.1 #taxa de aprendizagem
gama = 0.99 #fator de desconto (recompensas futuras)
epsilon = 0.1

# Ambiente de negociação
acoes = ['comprar','vender','manter'] #ações do agente
saldo_inicial = 1000  #saldo inicial
num_acoes_inicial = 0  #número inicial de ações

# Construindo a função de execução de uma ação (compra ou venda)

In [69]:
def executar(estado,acao,saldo,num_acoes,preco):
  if acao==0:
    if saldo>=preco:
      num_acoes+=1
      saldo-=preco

  elif acao==1:
    if num_acoes>0:
      num_acoes-=1
      saldo+=preco

  lucro = num_acoes*preco-saldo_inicial
  return(saldo,num_acoes,lucro)

# Treinando o agente segundo o protocolo epsilon-greedy

In [70]:
q_tabela = np.zeros((len(precos_treino),len(acoes)))

for n in range(num_episodios):
  saldo = saldo_inicial
  num_acoes = num_acoes_inicial

  for i, preco in enumerate(precos_treino[:-1]):

    estado = i

    # Ação adotada pelo agente por meio do esquema epsilon-greedy
    if np.random.random() < epsilon:
      acao = random.choice(range(len(acoes)))

    acao = np.argmax(q_tabela[estado])

    saldo,num_acoes,lucro = executar(estado,acao,saldo,num_acoes,preco)
    prox_estado = i+1

    q_tabela[estado][acao] += alpha * (lucro + gama * np.max(q_tabela[prox_estado]) - q_tabela[estado][acao])

    estado = prox_estado

# Aplicando o modelo aos últimos preços

In [71]:
# Aplicando o modelo

# Valores iniciais
saldo = saldo_inicial
num_acoes = num_acoes_inicial

# Aplicação do agente
for i, preco in enumerate(precos[:-1]):

    estado = i
    acao = np.argmax(q_tabela[estado])
    saldo,num_acoes,lucro = executar(estado,acao,saldo,num_acoes,preco)

# Venda de todas as ações
saldo = np.round(num_acoes*precos[len(precos)-1],2)

lucro = round(saldo-saldo_inicial,2)

if(lucro>=0):
  print(f'Começamos com um saldo inicial de {saldo_inicial} reais e fechamos com um saldo de {saldo} reais, totalizando um \
  lucro de {lucro} reais.')
else:
  print(f'Começamos com um saldo inicial de {saldo_inicial} reais e fechamos com um saldo de {saldo} reais, totalizando um\
  prejuízo de {abs(lucro)} reais.')

Começamos com um saldo inicial de 1000 reais e fechamos com um saldo de 957.16 reais, totalizando um  prejuízo de 42.84 reais.
